In [115]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time 
from random import randint
from datetime import date, timedelta
from datetime import datetime
from dateutil.parser import parse
import smtplib
from email.mime.multipart import MIMEMultipart
import re
import csv


csv_file = open('Flight_Data.csv', 'w', encoding='utf-8', newline='')
writer = csv.writer(csv_file)


def Start_HotWire(airport_from, airport_to, mid):
    
    airport_from = input('From which airport? ')
    airport_to = input('To which airport? ')
    mid = input('What is your departure date? Please use MM/DD/YYYY format only ')
    
    # Add and subtract 3 days from travel date named 'mid'
    mid = pd.to_datetime(mid)
    span = (pd.Timedelta('3 days'))
    end = mid+span
    start = mid-span
    day = (pd.Timedelta('1 days'))
    
    result = []
    
    while start <= end:
        date_span = "{date.month:02}/{date.day:02}/{date.year}".format(date=start)
        result.append(date_span)
        start += day
        
    # Generate all urls using the 1 week span around the expected travel date    
    for date in result:
        
        date_dict = {}
        
        urls = 'https://vacation.hotwire.com/Flights-Search?tmid=30276619572&trip=OneWay&leg1=from:'+airport_from+',to:'+airport_to+',departure:'+date+'TANYT&passengers=children:0,adults:1,seniors:0,infantinlap:Y&options=sortby:price&mode=search&paandi=true'
        
        driver = webdriver.Chrome()
        driver.get(urls)
        time.sleep(20)
        
       
    # Scrape information from each url    
        try:
            Airline = driver.find_elements_by_xpath('//span[@data-test-id="airline-name"]')
            Airline = list(map(lambda x: x.text, Airline[:5]))

            Duration = driver.find_elements_by_xpath('//span[@data-test-id="duration"]')
            Duration = list(map(lambda x: x.text, Duration[:5]))

            Price = driver.find_elements_by_xpath('//span[@data-test-id="listing-price-dollars"]')
            Price = list(map(lambda x: x.text, Price[:5]))
            

            Departure_and_Arrival_Time = driver.find_elements_by_xpath('//span[@class="medium-bold"]')
            Departure_and_Arrival_Time = list(map(lambda x: x.text, Departure_and_Arrival_Time[:5]))
            x = [item.split() for item in Departure_and_Arrival_Time]
            for i in x:
                Departure_time = i[0]
                Arrival_time = i[2]

            Stops = driver.find_elements_by_xpath('//span[@class="number-stops"]')
            Stops = list(map(lambda x: x.text, Stops[:5]))

            Layover_Info = driver.find_elements_by_xpath('//span[@data-test-id="layover-info"]')
            Layover_Info = list(map(lambda x: x.text, Layover_Info[:5]))
            y = [item.split() for item in Layover_Info]
            for x in y:
                i = x[4:9]
                i = (' '.join(i))
                    
         # Create dictionary for csv onput       
                date_dict['Date'] = date
                date_dict['Airline'] = Airline
                date_dict['Flight Duration'] = Duration            
                date_dict['Price'] = Price
                date_dict['Departure Time'] = Departure_time
                date_dict['Arrival Time'] = Arrival_time
                date_dict['Stops'] = Stops
                date_dict['Layover_Info'] = i
                
            writer.writerow(date_dict.values())
            
        except Exception as e:
            print(e)
            
            csv_file.close()
            driver.close()

            break
        


In [116]:
Start_HotWire('airport_from', 'airport_to', 'mid')

From which airport? jfk
To which airport? nrt
What is your departure date? Please use MM/DD/YYYY format only 10/21/2019
